In [1]:
import cv2
import keyboard
import pyautogui
# from model import load_model
from tensorflow.keras.models import load_model
import numpy as np
import time
import threading


In [2]:
# Load Gesture model
model = load_model("../Hand Gesture/handgesture1.hdf5") 
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 32)     │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 12, 12, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 4, 4, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 2, 2, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 2, 2, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 91,528 (357.54 KB)

 Trainable params: 91,526 (357.52 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [3]:
# # Import gesture label into pickle file
# file = open("Gesture_label.pickle","w")
# for i in range(0,6):
#     name = input("Name: ")
#     file.write(f"{name}\n")
# file.close()

In [4]:
# Load class name
f = open("Gesture_label.pickle","r")
labels = f.read().split('\n')
f.close()
print(labels)

['okay', 'paper', 'rock', 'scissor', 'thumbs', 'up', '']


In [5]:
def drawBox(frame, bbox):
    x, y, w, h = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
    cv2.rectangle(frame,(x, y), ((x + w), (y + h)), (255, 0, 255), 3, 1)
    # cv2.putText(frame, "Tracking" ,(75, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

In [6]:
def keybind(label):
    print('Keybind')
    ## Currently there is an error when you bind to arrow keys
    if label == 'okay':
        print('Play/Pause')
        pyautogui.hotkey('space')
        
    elif label == 'paper':
        print('Next Track')
        pyautogui.hotkey('ctrl', 'right')
        
    elif label == 'rock':
        print('Previous Track')
        pyautogui.hotkey('ctrl', 'left')
        
    elif label == 'scissor':
        print('Volume Up')
        pyautogui.hotkey('ctrl', 'up')
        
    elif label == 'thumbs':
        print('Volume Down')
        pyautogui.hotkey('ctrl', 'down')
        
    # elif label.name == '6':
    #     print('Shuffle Track')
    #     pyautogui.hotkey('ctrl', 's')
        
    elif label == 'up':
        print('Repeat Track')
        pyautogui.hotkey('ctrl', 'r')
    
    # elif key.name == 'm':
    #     print('Mute/Unmute')
    #     pyautogui.hotkey('ctrl', 'shift', 'down')  # This shortcut didn't work for some reasons...

    elif label == 'None':
        print("Invalid label")
        
    else: 
        print("Invalid label")

# ORIGINAL MODEL

In [7]:
# cap = cv2.VideoCapture(0)
# # Set-up Tracker
# tracker = cv2.legacy.TrackerMOSSE_create()
# ret, frame = cap.read()

# # # Bounding box
# bbox = cv2.selectROI("Tracking", frame, False)
# tracker.init(frame, bbox)

# # Inside your main loop
# while True:
#     ret, frame = cap.read()
#     if not ret:
#         print("Exception: Failed to initiate camera.")
#         break

#     timer = cv2.getTickCount()
#     ret, bbox = tracker.update(frame)

#     if ret:
#         drawBox(frame, bbox)
#     else:
#         cv2.putText(frame, "Lost", (75, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

#     fps = int(cv2.getTickFrequency() / (cv2.getTickCount() - timer))
#     cv2.putText(frame, str(fps), (75, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

#     x, y, w, h = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
#     roi = frame[y:y + h, x:x + w]

#     name_class = 'Unknown'  # Initialize with default value
#     if ret:
#         try:
#             # Image preprocessing
#             blur = cv2.GaussianBlur(roi, (5, 5), 0)
#             hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
#             mask = cv2.inRange(hsv, np.array([2, 50, 50]), np.array([20, 255, 255]))
#             filtered = cv2.GaussianBlur(mask, (3, 3), 0)
#             _, thresh = cv2.threshold(filtered, 127, 255, 0)
#             thresh = cv2.GaussianBlur(thresh, (5, 5), 0)
#             resized_thresh = cv2.resize(thresh, (60, 60))
#             input_image = cv2.merge([resized_thresh, resized_thresh, resized_thresh])
#             input_image = np.expand_dims(input_image, axis=0)

#             # Model prediction
            
#             pred = model.predict(input_image)
#             id_class = np.argmax(pred)
#             name_class = labels[id_class].capitalize()

#             cv2.imshow("Thresholded", thresh)
#         except Exception as e:
#             print(f"Error during prediction: {e}")
#             cv2.putText(frame, "Prediction Error", (75, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

#     # Display the class name on the frame
#     cv2.putText(frame, name_class, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    

#     # Normalize name_class to fit keybind function
#     name_class = name_class.lower()  # Convert to lowercase for consistency
#     # print(f"Predicted class: {name_class}")  # Debugging statement


#     # Fit prediction label with keybind
#     keybind(name_class)
    
#     # Display live video
#     cv2.imshow("Press 'q' to exit", frame)
#     key = cv2.waitKey(1)
    
#     if key == ord('q'):
#         print("Exiting the program.")
#         break

# # Release the camera and close all windows
# cap.release()
# cv2.destroyAllWindows()

# EXPERIMENTAL MODEL

In [17]:
def pred_label(roi):
    global name_class, thresh

    try:
        # Image processing
        blur = cv2.GaussianBlur(roi, (5, 5),0)
        hsv = cv2. cvtColor(blur, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, np.array([2, 50, 50]), np.array([20, 255, 255]))
        filtered = cv2.GaussianBlur(mask, (3, 3), 0)
        _, thresh = cv2.threshold(filtered, 127, 255, 0)
        thresh = cv2.GaussianBlur(thresh, (5, 5), 0)
        resized_thresh = cv2.resize(thresh, (60, 60))
        input_image = cv2.merge([resized_thresh, resized_thresh, resized_thresh])
        input_image = np.expand_dims(input_image, axis=0)
        
        # Check the presence of the hand
        hand_presence_threshold = 600  # Minimum number of non-zero pixels to consider as hand presence
        if cv2.countNonZero(thresh) < hand_presence_threshold:
            print("Hand not detected within the bounding box. Skipping prediction.")
            name_class = 'Unknown'
            return


        # Model prediction
        pred = model.predict(input_image)
        id_class = np.argmax(pred)
        name_class = labels[id_class].capitalize()

        # cv2.imshow("Thresholded", thresh)
    except Exception as e:
        print(f"Error during prediction: {e}")
        # cv2.putText(frame, "Prediction Error", (75, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)


In [9]:
# Function to handle keybinds in a separate thread
def run_keybind(label):
    global last_keybind_time
    global keybind_delay
    current_time = time.time()
    if current_time - last_keybind_time >= keybind_delay:
        keybind(label)
        last_keybind_time = current_time


In [10]:
# import time
# cap = cv2.VideoCapture(0)

# # Set-up Tracker
# tracker = cv2.legacy.TrackerMOSSE_create()
# ret, frame = cap.read()

# # # Bounding box
# bbox = cv2.selectROI("Tracking", frame, False)
# tracker.init(frame, bbox)



# # Global variable to hold the latest class name
# name_class = 'Unknown'  

# # Define a variable to hold the thread reference
# prediction_thread = None
# keybind_thread = None
# keybind_delay = 3
# last_keybind_time = 0

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         print("Exception: Failed to initiate camera.")
#         break

#     timer = cv2.getTickCount()
#     ret, bbox = tracker.update(frame)

#     if ret:
#         drawBox(frame, bbox)
#     else:
#         cv2.putText(frame, "Lost", (75, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

#     fps = int(cv2.getTickFrequency() / (cv2.getTickCount() - timer))
#     cv2.putText(frame, str(fps), (75, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

#     x, y, w, h = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
#     roi = frame[y:y + h, x:x + w]

#     # Start a new thread for prediction if the previous one has finished
#     if (prediction_thread is None) or (not prediction_thread.is_alive()):
#         prediction_thread = threading.Thread(target=pred_label, args=(roi,))
#         prediction_thread.start()

#     # Display the class name on the frame
#     cv2.putText(frame, name_class, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

#     # Normalize name_class to fit keybind function
#     name_class_normalized = name_class.lower()

#     # Run keybind in a separate thread to avoid blocking the main loop
#     if (keybind_thread is None) or (not keybind_thread.is_alive()):
#         keybind_thread = threading.Thread(target=run_keybind, args=(name_class_normalized,))
#         keybind_thread.start()

#     # Display live video
#     cv2.imshow("Press 'q' to exit", frame)
#     key = cv2.waitKey(1)

#     if key == ord('q'):
#         print("Exiting the program.")
#         break

# # Release the camera and close all windows
# cap.release()
# cv2.destroyAllWindows()


# CHATGPT GOI I

In [11]:
# import cv2
# import threading
# import time
# import numpy as np
# import pyautogui  # Make sure pyautogui is installed and imported

# # Assume the necessary imports like model and labels are already done

# cap = cv2.VideoCapture(0)

# # Set-up Tracker
# tracker = cv2.legacy.TrackerMOSSE_create()
# ret, frame = cap.read()

# # # Bounding box
# bbox = cv2.selectROI("Tracking", frame, False)
# tracker.init(frame, bbox)

# # Global variables
# name_class = 'Unknown'  
# prediction_thread = None
# keybind_thread = None
# last_keybind_time = 0
# keybind_delay = 1  # Minimum delay between keybind actions in seconds

# # Your provided keybind function
# def keybind(label):
#     print('Keybind')
#     if label == 'okay':
#         print('Play/Pause')
#         pyautogui.hotkey('space')
        
#     elif label == 'paper':
#         print('Next Track')
#         pyautogui.hotkey('ctrl', 'right')
        
#     elif label == 'rock':
#         print('Previous Track')
#         pyautogui.hotkey('ctrl', 'left')
        
#     elif label == 'scissor':
#         print('Volume Up')
#         pyautogui.hotkey('ctrl', 'up')
        
#     elif label == 'thumbs':
#         print('Volume Down')
#         pyautogui.hotkey('ctrl', 'down')
        
#     elif label == 'up':
#         print('Repeat Track')
#         pyautogui.hotkey('ctrl', 'r')
    
#     else: 
#         print("Invalid label")

# # Function to handle predictions
# def pred_label(roi):
#     global name_class
#     try:
#         # Image preprocessing
#         blur = cv2.GaussianBlur(roi, (5, 5), 0)
#         hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
#         mask = cv2.inRange(hsv, np.array([2, 50, 50]), np.array([20, 255, 255]))
#         filtered = cv2.GaussianBlur(mask, (3, 3), 0)
#         _, thresh = cv2.threshold(filtered, 127, 255, 0)
#         thresh = cv2.GaussianBlur(thresh, (5, 5), 0)
#         resized_thresh = cv2.resize(thresh, (60, 60))
#         input_image = cv2.merge([resized_thresh, resized_thresh, resized_thresh])
#         input_image = np.expand_dims(input_image, axis=0)

#         # Model prediction
#         pred = model.predict(input_image)
#         id_class = np.argmax(pred)
#         name_class = labels[id_class].capitalize()

#         cv2.imshow("Thresholded", thresh)
#     except Exception as e:
#         print(f"Error during prediction: {e}")

# # Function to handle keybinds in a separate thread
# def run_keybind(label):
#     global last_keybind_time
#     current_time = time.time()
#     if current_time - last_keybind_time >= keybind_delay:
#         keybind(label)
#         last_keybind_time = current_time

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         print("Exception: Failed to initiate camera.")
#         break

#     timer = cv2.getTickCount()
#     ret, bbox = tracker.update(frame)

#     if ret:
#         drawBox(frame, bbox)
#     else:
#         cv2.putText(frame, "Lost", (75, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

#     fps = int(cv2.getTickFrequency() / (cv2.getTickCount() - timer))
#     cv2.putText(frame, str(fps), (75, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

#     x, y, w, h = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
#     roi = frame[y:y + h, x:x + w]

#     # Start a new thread for prediction if the previous one has finished
#     if (prediction_thread is None) or (not prediction_thread.is_alive()):
#         prediction_thread = threading.Thread(target=pred_label, args=(roi,))
#         prediction_thread.start()

#     # Display the class name on the frame
#     cv2.putText(frame, name_class, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

#     # Normalize name_class to fit keybind function
#     name_class_normalized = name_class.lower()

#     # Run keybind in a separate thread to avoid blocking the main loop
#     if (keybind_thread is None) or (not keybind_thread.is_alive()):
#         keybind_thread = threading.Thread(target=run_keybind, args=(name_class_normalized,))
#         keybind_thread.start()

#     # Display live video
#     cv2.imshow("Press 'q' to exit", frame)
#     key = cv2.waitKey(1)

#     if key == ord('q'):
#         print("Exiting the program.")
#         break

# # Release the camera and close all windows
# cap.release()
# cv2.destroyAllWindows()


# EXPERIMENTAL CODE 2

In [15]:
import time
cap = cv2.VideoCapture(0)

# Global variable to hold the latest class name
name_class = 'Unknown'  

# Define a variable to hold the thread reference
prediction_thread = None
keybind_thread = None
keybind_delay = 3
last_keybind_time = 0

# Bounding box coordinate
x, y, w, h = 30, 80, 250, 200

while True:
    ret, frame = cap.read()
    if not ret:
        print("Exception: Failed to initiate camera.")
        break

    timer = cv2.getTickCount()


    fps = int(cv2.getTickFrequency() / (cv2.getTickCount() - timer))
    cv2.putText(frame, str(fps), (75, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Display bounding box
    roi = frame[y:y + h, x:x + w]
    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Start a new thread for prediction if the previous one has finished
    if (prediction_thread is None) or (not prediction_thread.is_alive()):
        prediction_thread = threading.Thread(target=pred_label, args=(roi,))
        prediction_thread.start()

    # Display the class name on the frame
    cv2.putText(frame, name_class, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Normalize name_class to fit keybind function
    name_class_normalized = name_class.lower()

    # Run keybind in a separate thread to avoid blocking the main loop
    if (keybind_thread is None) or (not keybind_thread.is_alive()):
        keybind_thread = threading.Thread(target=run_keybind, args=(name_class_normalized,))
        keybind_thread.start()

    # Display live video
    cv2.imshow("Press 'q' to exit", frame)
    key = cv2.waitKey(1)

   # Display the thresholded feed if thresh is available
    if thresh is not None:
        cv2.imshow("Thresholded Feed", thresh)

        
    if key == ord('q'):
        print("Exiting the program.")
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()


Keybind
Invalid label
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 